In [74]:
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import json

In [75]:
id_li= [] # dictionary will store id and information about top games

In [76]:
def scrape_bgg_page(page_num=1):
    global id_li
    
    page_num = str(page_num)
    
    initial_url = 'https://boardgamegeek.com/browse/boardgame/page/{0}?sort=rank&sortdir=asc'.format(page_num)
    # obtain table of top games from bgg
    r = requests.get(initial_url)
    soup = BeautifulSoup(r.text, "html.parser")  
    rank_table = soup.find_all("tr", attrs={"id": "row_"})
    
    # Loop through all 100 items and store info in dictionary
    for idx, elem in enumerate(rank_table):

        # Find the main information about a game
        links = elem.find_all("a", {'class':'primary'})

        # Obtain the game's url
        url = links[0]['href']

        # From that url, obtain the game id
        id_ = re.findall(r'[0-9]+', url)[0]

        # Also return the title of the game
        url = url.replace('/boardgame/', '')
        url = url.replace(id_, '')
        url = url.replace('/', '')
        url = url.replace('-','_')

        # Finally, obtain the release year
        year_ = elem.find_all('span')
        year_ = year_[0].contents
        year_ = re.findall(r'[0-9]+', str(year_))[0]
        id_li.append([id_, url, year_])

In [77]:
# Scrape top 10 games
for i in range(1,11):
    scrape_bgg_page(i)

In [ ]:
id_li

In [79]:
df = pd.DataFrame(id_li, columns = ['id', 'title', 'year'])

,id,title,year
0,174430,gloomhaven,2017
1,161936,pandemic_legacy_season_1,2015
2,224517,brass_birmingham,2018
3,167791,terraforming_mars,2016
4,291457,gloomhaven_jaws_lion,2020
...,...,...,...
995,181796,prodigals_club,2015
996,121297,fleet,2012
997,234669,legacy_dragonholt,2017
998,50768,ninjato,2011


In [81]:
df['id'] = df['id'].astype(int)
df['year'] = df['year'].astype(int)

In [82]:
df.to_csv('top_games.csv')